## Stackline Onboarding Data Analyst Assessment
**Author:** Elena Burlando

## Overview
The "Data" tab contains weekly sales and price data for K-Cups (a sub-category of Coffee) for calendar year 2017.
Each row represents a specific SKU for a specific week.
Assume that a single K-cup is equal to a single serving of coffee. For example, SKU: B007KPFTNG contains 2 boxes of 24 K-Cups. This is equivalent to 48 K-Cups and therefore 48 servings.

The purpose of this assessment is to gauge your 1) technical skills, 2) market/trend analysis skills, and 3) data quality/auditing skills.

## Business Problem

1) Find the number of servings for each SKU by extracting information from each SKU Title. The calculations and formulas used to get to the number of servings should be show in addition columns on the data tab.

2) Assume you have a meeting with Stackline Engineering tomorrow. The Engineering team needs to know what, if any, problems/inconsistencies/etc. exist within the K-Cup sub-category. What would you tell the Engineering team during tomorrow's meeting? Any problems or inconsistencies you find within the data should be outlined, along with an example from the data set, in a new tab labeled 'Data Inconsistencies'.

### EDA (Exploratory Data Analysis)

In [1]:
# import libraries 

import numpy as np
import pandas as pd

In [2]:
#import data
pd.set_option('display.max_colwidth', None)

df = pd.read_excel(r'Excel - Assessment.xlsx', 'Data')
print(df.shape)
print(df.info())
df.head()

(157499, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157499 entries, 0 to 157498
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SKU              157499 non-null  object 
 1   Title            157499 non-null  object 
 2   Brand            157499 non-null  object 
 3   Parent Category  157499 non-null  object 
 4   Sub-Category     157499 non-null  object 
 5   WeekID           157499 non-null  int64  
 6   Retail Price     157499 non-null  float64
 7   Sales            157499 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 9.6+ MB
None


,SKU,Title,Brand,Parent Category,Sub-Category,WeekID,Retail Price,Sales
0,B01MYT8VBS,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",A Great Surprise,Coffee,K Cups,202252,26.38,395.70
1,B0007YHARK,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",Aloha Island Coffee,Coffee,K Cups,202252,26.14,26.14
2,B00KEVJF88,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",Aloha Island Coffee,Coffee,K Cups,202252,32.00,96.00
3,B00KLDHFV8,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",Aloha Island Coffee,Coffee,K Cups,202252,47.95,47.95
4,B00KOS3Z96,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",Aloha Island Coffee,Coffee,K Cups,202252,51.97,51.97


In [3]:
# basic statistics
df.describe()

,WeekID,Retail Price,Sales
count,157499.000000,157499.000000,157499.000000
mean,202228.009086,33.786037,2528.488399
std,14.458184,36.365680,10655.954612
min,202201.000000,2.190000,0.000000
25%,202216.000000,16.820000,86.860000
50%,202228.000000,27.820000,360.230000
75%,202240.000000,43.990000,1415.615000
max,202252.000000,10698.930000,706216.280000


Notes from basic statistics of numeric data: 
- 'WeekID' goes from 01 to 52 (annual number of weeks). The first four digits are most likely indicating a year which will be year 2022. The project overview is indicating that this data is for calendar year 2017. 
- 'Retail Price' ranges from $2.19$ US dollars to $10,698.93$ US dollars with standard deviation of $36.37$ US dollars and mean of $33.79$ US dollars.  
- Min 'Sales' is $0.00$ US dollars could be a missing value. Need to look into sales with $0.00$ US dollars values
- Mean and median of 'Sales' is drastically different. Max of 'Sales' is $706,216.28$ US dollars when the median is $360.23$ US dollars. 


In [4]:
#describe categorical columns
df.describe(include = ['O'])

,SKU,Title,Brand,Parent Category,Sub-Category
count,157499,157499,157499,157499,157499
unique,4763,4572,136,1,1
top,B01C3EG9LO,Starbucks Sumatra Coffee K-Cups,Green Mountain,Coffee,K Cups
freq,52,207,17670,157499,157499


Notes from the basic statistics on categorical data: 
- total of 136 brands of coffee are presented
- top seller is Starbucks Sumatra K-Cups by Green Mountain brand
- Unique 'SKU' and unique 'Title' don't match (191 more unique SKUs). Need to look further into it.
- Columns 'Parent Category' and 'Sub-Category' have only one unique value and can be disregarded in analysis. 

In [5]:
# checking if there are any missing values
df.isnull().sum()

SKU                0
Title              0
Brand              0
Parent Category    0
Sub-Category       0
WeekID             0
Retail Price       0
Sales              0
dtype: int64

#### Entries with $0.0 in Sales

In [6]:
from pandasql import sqldf

In [7]:
pysqldf = lambda q: sqldf(q, globals())

In [8]:
q = """SELECT
        sku, title, sales
     FROM
        df 
    WHERE sales = 0.0 ;"""

In [9]:
orders_with_zero_sales = pysqldf(q)
orders_with_zero_sales

,SKU,Title,Sales
0,B0728K6CBW,"Starbucks Peppermint Mocha Flavored Single Cup Coffee for Keurig Brewers, 4 Boxes of 6 (24 Total K-Cup Pods)",0.0
1,B00AXO9NRO,Donut Shop Decaf KCup Coffee 44 Count,0.0
2,B005ENWLC8,"Green Mountain Coffee, Kenya Highlands Keurig K-Cup Pods (24 count)",0.0
3,B009QMA184,San Francisco Bay - Hazelnut Creme - 72 Cups,0.0
4,B00CX1KOJQ,Brooklyn Beans Fuhgeddaboutit KCups - 24ct Box,0.0
...,...,...,...
74,B00C6PNJ3M,DIE60051012 - Green Mountain Coffee Roasters Butter Toffee Coffee K-Cups,0.0
75,B009A50LLE,LAV0425 - Aroma Point Espresso Cartrdg,0.0
76,B009QMA184,San Francisco Bay - Hazelnut Creme - 72 Cups,0.0
77,B010OCEEHI,Crazy Cups Frosted Oatmeal Cookie Flavored Coffee Single Serve Cups (88 Count),0.0


79 rows are missing sales data. 

#### Getting Count of K-Cups

In order to find the number of servings we need to retrieve that data from 'Title' column. 'Title' data is an object data consisting of both text and numeric values. It is not standardized so number of servings can be recorded in one row as a number of K-cups with the mention of the number of packs or as a total or count in the other row, or there are some cases of the coffee weight with no information on the number of servings. 

With the respect of above observations, I'm planning to do the following steps: 
1. Clean up 'Title' column data by getting rid of punctuations and converting it to lowercase. 
2. Split the data into separate words and aggregate the words so we can visually inspect the words to find keywords associated with servings information. 
3. Assign keywords to three groups: single_cup_keywords, box_keywords, and total_keywords. single_cup_keywords indicates how many servings per pack, box_keywords indicates number of boxes, total_keywords indicates total number of servings per SKU. 
4. Parse the cleaned data of cleaned 'Title' column with Python regular expressions library to find numeric values for associated keywords. 
5. Create columns with grouped keywords values and perform additional calculations to find total servings per SKU weekly order. 


**1. Cleaning 'Title' column data**

In [10]:
# import regular expressions library to get servings data from 'Title' column 
import re

In [11]:
# convert 'SKU', 'Title', 'Brand' columns from object to string
df[['SKU', 'Title', 'Brand']] = df[['SKU', 'Title', 'Brand']].astype('string')

In [12]:
# ensure convertion completed successfully
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157499 entries, 0 to 157498
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SKU              157499 non-null  string 
 1   Title            157499 non-null  string 
 2   Brand            157499 non-null  string 
 3   Parent Category  157499 non-null  object 
 4   Sub-Category     157499 non-null  object 
 5   WeekID           157499 non-null  int64  
 6   Retail Price     157499 non-null  float64
 7   Sales            157499 non-null  float64
dtypes: float64(2), int64(1), object(2), string(3)
memory usage: 9.6+ MB


In [13]:
# prior to parsing through 'Title' column data, I will clean it to make parsing easier

# convert % sign to 'percent' word as I will get rid of all punctuation in line 6. But since % usually follows 
# a number we don't want it to be confused with servings which could happen if we completely get rid of % sign 
# without converting it to a word. Also converting all text to lowercase. 
df['Title_clean'] = df['Title'].str.replace('%',' percent').str.lower()

# converting '-' to ' ' to address examples like this '24-count'
df['Title_clean'] = df['Title_clean'].str.replace('-', ' ')
# get rid of all punctuation
df['Title_clean'] = df['Title_clean'].str.replace('[^\w\s]',' ')
# get rid of double spaces so they won't appear after splitting as separate words
df['Title_clean'] = df['Title_clean'].str.replace('  ',' ')

df.head(10)

,SKU,Title,Brand,Parent Category,Sub-Category,WeekID,Retail Price,Sales,Title_clean
0,B01MYT8VBS,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",A Great Surprise,Coffee,K Cups,202252,26.38,395.70,cake boss coffee k cups variety pack single serve for keurig brewers 36 count
1,B0007YHARK,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",Aloha Island Coffee,Coffee,K Cups,202252,26.14,26.14,12 k cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend
2,B00KEVJF88,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",Aloha Island Coffee,Coffee,K Cups,202252,32.00,96.00,12 single serve cups of 100 percent pure kona coffee for use in keurig k cup brewing systems exclusive private reserve diamond kona one cups medium roast box of 12 cups
3,B00KLDHFV8,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",Aloha Island Coffee,Coffee,K Cups,202252,47.95,47.95,24 single serve kona one cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend
4,B00KOS3Z96,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",Aloha Island Coffee,Coffee,K Cups,202252,51.97,51.97,24 single serve kona one cups of water process decaf organic arabica tropical coffee for keurig k cup brewing systems from aloha island coffee
5,B00KEVJGSM,"100% Pure Kona Single-Serve Cups for Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Medium Roast Kona-One-Cups, Box of 20 Single-Serve Cups",Aloha Island Coffee,Coffee,K Cups,202252,50.88,101.76,100 percent pure kona single serve cups for keurig k cup brewing systems exclusive private reserve diamond medium roast kona one cups box of 20 single serve cups
6,B00KM8OLFA,"12 Kone-One-Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",Aloha Island Coffee,Coffee,K Cups,202252,27.49,27.49,12 kone one cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems
7,B00KOS3Y74,"12 Single-Serve Cups of Water Process Decaf Organic Arabica Coffee, for Use With Keurig K-cup Brewing Systems, From Aloha Island Coffee",Aloha Island Coffee,Coffee,K Cups,202252,29.68,29.68,12 single serve cups of water process decaf organic arabica coffee for use with keurig k cup brewing systems from aloha island coffee
8,B00TBVYB4A,"12 Kona-One-Cups of Kings Reserve Kona Hawaiian DIAMOND Coffee Blend, Medium Roast",Aloha Island Coffee,Coffee,K Cups,202252,21.75,108.75,12 kona one cups of kings reserve kona hawaiian diamond coffee blend medium roast
9,B00KM8OMIG,"24 Single-Serve Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",Aloha Island Coffee,Coffee,K Cups,202252,48.95,48.95,24 single serve cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems


In [14]:
# creating a df with unique titles only for more efficient parsing 
# (there is 4,572 unique titles vs 157,499 rows of data)
df_title_unique = df.drop_duplicates(subset=['Title'], \
                                          keep='first', ignore_index=True)
df_title_unique = df_title_unique[['Title', 'Title_clean']]
df_title_unique

,Title,Title_clean
0,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",cake boss coffee k cups variety pack single serve for keurig brewers 36 count
1,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",12 k cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend
2,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",12 single serve cups of 100 percent pure kona coffee for use in keurig k cup brewing systems exclusive private reserve diamond kona one cups medium roast box of 12 cups
3,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",24 single serve kona one cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend
4,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",24 single serve kona one cups of water process decaf organic arabica tropical coffee for keurig k cup brewing systems from aloha island coffee
...,...,...
4567,"Italian Roast K-Cup Variety Pack, Keurig 2.0 K Cup Coffee Sampler with Adagio Toscana Blend, Faro Italian, Lavazza Gran Selezione & Tully's Italian Roast Extra Bold, Keurig 2.0 K-Cups (96 Count)",italian roast k cup variety pack keurig 2 0 k cup coffee sampler with adagio toscana blend faro italian lavazza gran selezione tully s italian roast extra bold keurig 2 0 k cups 96 count
4568,"Green Mountain Coffee WccIm K-Cup Portion Count for Keurig K-Cup Brewers, Caramel Vanilla Cream, 24 Count (Pack of 2) GvlSZ",green mountain coffee wccim k cup portion count for keurig k cup brewers caramel vanilla cream 24 count pack of 2 gvlsz
4569,Keurig Green Mountain HzrVf Coffee K Cup Packs - Breakfast Blend - 72 Count (2 Pack) IYcfv,keurig green mountain hzrvf coffee k cup packs breakfast blend 72 count 2 pack iycfv
4570,"Green Mountain Coffee ixILa K-Cup Portion Count for Keurig K-Cup Brewers, Caramel Vanilla Cream, 24 Count (Pack of 2) QOBEQ",green mountain coffee ixila k cup portion count for keurig k cup brewers caramel vanilla cream 24 count pack of 2 qobeq


**2. Finding Keywords**

In [15]:
# separate the words of 'Title' column through split function so we can aggregate into a DataFrame 
# and analyze the words to see if we would need to address additional boundary cases 
title_words = df_title_unique['Title_clean'].map(lambda x: {c:1 for c in x.split(' ')}).tolist()
title_words[:5]

[{'cake': 1,
  'boss': 1,
  'coffee': 1,
  'k': 1,
  'cups': 1,
  'variety': 1,
  'pack': 1,
  'single': 1,
  'serve': 1,
  'for': 1,
  'keurig': 1,
  'brewers': 1,
  '36': 1,
  'count': 1},
 {'12': 1,
  'k': 1,
  'cups': 1,
  'of': 1,
  'breakfast': 1,
  'blend': 1,
  'certified': 1,
  'organic': 1,
  'arabica': 1,
  'tropical': 1,
  'coffee': 1,
  'surf': 1,
  's': 1,
  'up': 1,
  'get': 1,
  'your': 1,
  'day': 1,
  'started': 1},
 {'12': 1,
  'single': 1,
  'serve': 1,
  'cups': 1,
  'of': 1,
  '100': 1,
  'percent': 1,
  'pure': 1,
  'kona': 1,
  'coffee': 1,
  'for': 1,
  'use': 1,
  'in': 1,
  'keurig': 1,
  'k': 1,
  'cup': 1,
  'brewing': 1,
  'systems': 1,
  'exclusive': 1,
  'private': 1,
  'reserve': 1,
  'diamond': 1,
  'one': 1,
  'medium': 1,
  'roast': 1,
  'box': 1},
 {'24': 1,
  'single': 1,
  'serve': 1,
  'kona': 1,
  'one': 1,
  'cups': 1,
  'of': 1,
  'breakfast': 1,
  'blend': 1,
  'certified': 1,
  'organic': 1,
  'arabica': 1,
  'tropical': 1,
  'coffee': 1,
  

In [16]:
# convert a list of dictionaries to DataFrame
title_words_df = pd.DataFrame(title_words).fillna(0).astype(int)
title_words_df.head()

,cake,boss,coffee,k,cups,variety,pack,single,serve,for,...,ytnls,sharv,adagio,toscana,wccim,gvlsz,hzrvf,iycfv,ixila,qobeq
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# visually review words that might be used as keywords associated with the number of servings and 
# words/values that could be boundary cases
pd.set_option('display.max_rows', 1540)
title_words_df.sum()

cake                          50
boss                          24
coffee                      3488
k                           3519
cups                        1992
variety                      353
pack                        1267
single                      1756
serve                       1432
for                         1572
keurig                      2534
brewers                     1644
36                           180
count                       3072
12                           619
of                          1022
breakfast                    255
blend                        968
certified                     24
organic                      263
arabica                       40
tropical                       4
surf                           2
s                            645
up                            14
get                            7
your                           5
day                            2
started                        2
100                          211
percent   

*** 
**Potential Keywords:** 
- cups and cup 
- k cups (it could be singular)
- kcups
- count
- single serve
- serve
- total 
- pod
- bulk
- net
- 4boxes   
- carton
- cluster
- selection
- pack
- x
- ea
- each
- singles
- ekocups
- units
- lids
- set
- cans
- 88ct
- blnd16ct
- onecups 

***

**Examples of boundary cases:** 
- 49oz, 1kg, 162g
- 33059, 00732, 108879, 114685, 114686, 114014, 01800, 15508, 01803, 1000  large numbers that don't fit the serving size (need to look at it individually)
- v1
- wgp016445
- 2x, 75x, 2x18
- 2pc
- 7lwgkdx
- 4x24
- 1x16oz
- 2016 - year
- sh4f
- 49th
- kiteveen91gmt6503, die60051012, gmt7792ct

In [18]:
df_title_unique.loc[df_title_unique['Title_clean'].str.contains('33059', '00732')]

,Title,Title_clean
4384,San Francisco Bay Fog Chaser 36 Ct Box - 33059 - Biodegradable K cups - 100% Organic,san francisco bay fog chaser 36 ct box 33059 biodegradable k cups 100 percent organic


**3. Assign keywords to three groups: single_cup_keyword (count), box_keyword (box), and total_keyword (total).**

In [19]:
single_cup_keywords = ['cup', 'k cup', 'kcup', 'single serve', 'serve', 'pod', 'cluster', 'ea',
                       'singles', 'single', 'ekocup', 'unit','lid', 'can', 'onecup', 'pack of...']
box_keywords = ['box', '...pack', 'carton', 'set']
total_keywords = ['count', 'ct', 'total', 'net', 'total of' ] 


In [20]:
# checking if we need to include words such as 'bulk' and 'selection' to keyword groups
for row in df_title_unique['Title_clean']:
    match = re.search("\w\S*.bulk.*\w", row) 
    if match is not None: 
        print(match)
    else: 
        pass

<re.Match object; span=(109, 129), match='count bulk value bag'>


In [21]:
for row in df_title_unique['Title_clean']:
    match = re.search("\w\S*.selection.*\w", row) 
    if match is not None: 
        print(match)
    else: 
        pass

<re.Match object; span=(31, 106), match='s selection single serve cups for keurig k cup br>
<re.Match object; span=(31, 114), match='s selection and flavored lovers single serve cups>
<re.Match object; span=(57, 81), match='count selection may vary'>
<re.Match object; span=(81, 139), match='large selection of house and breakfast blends k c>
<re.Match object; span=(8, 66), match='gourmet selections vanilla biscotti coffee k cups>
<re.Match object; span=(8, 82), match='gourmet selections single cup for keurig brewers >
<re.Match object; span=(8, 80), match='gourmet selections coffee black silk for keurig b>
<re.Match object; span=(8, 58), match='gourmet selections caramel drizzle k cups 48 coun>
<re.Match object; span=(8, 58), match='gourmet selections caramel drizzle k cups 72 coun>
<re.Match object; span=(8, 58), match='gourmet selections caramel drizzle k cups 96 coun>
<re.Match object; span=(8, 63), match='gourmet selections classic roast coffee 120 k cup>
<re.Match object; span=(9, 

***
Both words: 'bulk' and 'selection' don't need to be included to keyword groups as they are not associated with any servings. 
***

**4. Parsing data**

***
**4.1 Single_cup keywords:**

single_cup_keywords = ['cup', 'k cup', 'kcup', 'single serve', 'serve', 'pod', 'cluster', 'ea',
                       'single', 'ekocup', 'unit','lid', 'can', 'onecup']
***

In [22]:
# Since 'count' word has been mentioned the most, we will use it as the main keyword to search for 
# to retrieve numeric value associated with it. If 'count' not found, we will move to other single_cup keywords

count_list = []

for row in df_title_unique['Title_clean']:
    match = re.search("\d\S*.count|\d\S*.ct", row) 
    if match is not None: 
        match_num = re.search(r'\d+', match.group())
        count_list.append(match_num.group())
    else: 
        match = re.search("\d\S*.k cup|\d\S*.cup|\d\S*.single serv|\d\S*.kon|count of.*\S\d|\d\S*.capsule", row)
        if match is not None: 
            match_num = re.search(r'\d+', match.group())
            count_list.append(match_num.group())
        else: 
            count_list.append(0)
    

In [23]:
# create a column 'count' with number of servings associated with single_cup_keywords

pd.set_option('display.max_rows', 100)
df_title_unique['count'] = count_list
df_title_unique['count'] = df_title_unique['count'].astype(str).astype(int)
df_title_unique.head(100)

,Title,Title_clean,count
0,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",cake boss coffee k cups variety pack single serve for keurig brewers 36 count,36
1,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",12 k cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,12
2,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",12 single serve cups of 100 percent pure kona coffee for use in keurig k cup brewing systems exclusive private reserve diamond kona one cups medium roast box of 12 cups,12
3,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",24 single serve kona one cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,24
4,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",24 single serve kona one cups of water process decaf organic arabica tropical coffee for keurig k cup brewing systems from aloha island coffee,24
5,"100% Pure Kona Single-Serve Cups for Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Medium Roast Kona-One-Cups, Box of 20 Single-Serve Cups",100 percent pure kona single serve cups for keurig k cup brewing systems exclusive private reserve diamond medium roast kona one cups box of 20 single serve cups,20
6,"12 Kone-One-Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",12 kone one cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems,12
7,"12 Single-Serve Cups of Water Process Decaf Organic Arabica Coffee, for Use With Keurig K-cup Brewing Systems, From Aloha Island Coffee",12 single serve cups of water process decaf organic arabica coffee for use with keurig k cup brewing systems from aloha island coffee,12
8,"12 Kona-One-Cups of Kings Reserve Kona Hawaiian DIAMOND Coffee Blend, Medium Roast",12 kona one cups of kings reserve kona hawaiian diamond coffee blend medium roast,12
9,"24 Single-Serve Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",24 single serve cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems,24


In [24]:
# checking the values
df_title_unique['count'].unique()

array([   36,    12,    24,    20,    40,   100,    50,    18,     0,
          80,    96,    48,    72,   120,    32,    10,    84,   160,
          16,    30,    64,   108,    52,    56,    60,    22,    44,
          88,     4,     8,     2,    54,   162,   144,   192,    19,
          42,   180,     6,     9,  1000, 15508,  7792,   110,    15,
         200,   240,   119,   176,    75,    70,    82,    62,     3,
         128,    66,    13,   132,   270,    25,    35,    90])

Values such as 1000, 15508,  7792 are most likely associated with something else rather than single_cup_keyword.

In [25]:
# Let's take a look at each entry with this value
df_title_unique.loc[df_title_unique['count'] == 1000]

,Title,Title_clean,count
1452,"Gevalia Branded Hot Cups, 16 Ounce, 1000 Count",gevalia branded hot cups 16 ounce 1000 count,1000


1000 cups is a true count.

In [44]:
# Let's take a look at each entry with this value
df_title_unique.loc[df_title_unique['count'] == 15508]

,Title,Title_clean,count,box,total
1535,"Keurig 15508 K-Cup Mini-Brewers, Green Mountain Nantucket Blend-18 cups",keurig 15508 k cup mini brewers green mountain nantucket blend 18 cups,15508,1,15508


In [46]:
# Number 15508 is referring here to Keurig coffeemaker serial number. 
# Let's assign correct '18' cups value instead.

df_title_unique.loc[1535, 'count'] = 18
df_title_unique.loc[1535, 'count']

18

In [47]:
# Let's take a look at each entry with this value
df_title_unique.loc[df_title_unique['count'] == 7792]

,Title,Title_clean,count,box,total
1586,GMT7792CT - Green Mountain Coffee Roasters Hazelnut Decaf Coffee K-Cups,gmt7792ct green mountain coffee roasters hazelnut decaf coffee k cups,7792,1,7792


In [48]:
# 7792 is a part of an item # and not the count. 
# Let's change it to 0 value since there is no indication of the number of cups

df_title_unique.loc[1586, 'count'] = 0
df_title_unique.loc[1586, 'count']

0

In [49]:
# checking if there is any other errors on the number of servings above 200
df_title_unique.loc[df_title_unique['count'] > 200]

,Title,Title_clean,count,box,total
1452,"Gevalia Branded Hot Cups, 16 Ounce, 1000 Count",gevalia branded hot cups 16 ounce 1000 count,1000,1,1000
2033,"Kirkland Pacific Bold K-Cups, (240 K Cups)",kirkland pacific bold k cups 240 k cups,240,1,240
2034,Kirkland Pacific Bold K-Cups (240 Count),kirkland pacific bold k cups 240 count,240,1,240
2036,"Kirkland Signature Coffee Single Serve K-Cup (Breakfast Blend, 240 K-Cups)",kirkland signature coffee single serve k cup breakfast blend 240 k cups,240,1,240
3326,"San Francisco Bay OneCup, French Roast, 240 Count- Single Serve Coffee,VBNF Compatible with Keurig K-cup Brewers",san francisco bay onecup french roast 240 count single serve coffee vbnf compatible with keurig k cup brewers,240,1,240
3488,"San Francisco Bay OneCup, Breakfast Blend, 240 Count- Single Serve Coffee, SDFN Compatible with Keurig K-cup Brewers",san francisco bay onecup breakfast blend 240 count single serve coffee sdfn compatible with keurig k cup brewers,240,1,240
3534,Folgers Classic Decaf Medium Roast - Makes 270 cups - 33.9oz (2lb 1.9oz) 961g,folgers classic decaf medium roast makes 270 cups 33 9oz 2lb 1 9oz 961g,270,1,270


*** 
**4.2 Box keywords:**

box_keywords = ['box', 'pack', 'carton', 'set']
***

In [27]:
# Since 'box' and '..pack' word has been mentioned the most, we will use it as the main keyword to search for 
# to retrieve numeric value associated with it. If 'box' not found, we will move to other box keywords: carton, 
# set, and pack of. 

box_list = []

for row in df_title_unique['Title_clean']:
    match = re.search("\d\S*.box|\d\S*.pack", row) 
    if match is not None: 
        match_num = re.search(r'\d+', match.group())
        box_list.append(match_num.group())
    else: 
        match = re.search("\d\S*.carton|pack of.\S\d|\d\S*.set", row)
        if match is not None: 
            match_num = re.search(r'\d+', match.group())
            box_list.append(match_num.group())
        else: 
            box_list.append(0)

In [28]:
# create a column 'box' with number of boxes associated with box_keywords

df_title_unique['box'] = box_list
df_title_unique['box'] = df_title_unique['box'].astype(str).astype(int)
df_title_unique.head(100)

,Title,Title_clean,count,box
0,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",cake boss coffee k cups variety pack single serve for keurig brewers 36 count,36,0
1,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",12 k cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,12,0
2,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",12 single serve cups of 100 percent pure kona coffee for use in keurig k cup brewing systems exclusive private reserve diamond kona one cups medium roast box of 12 cups,12,0
3,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",24 single serve kona one cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,24,0
4,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",24 single serve kona one cups of water process decaf organic arabica tropical coffee for keurig k cup brewing systems from aloha island coffee,24,0
5,"100% Pure Kona Single-Serve Cups for Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Medium Roast Kona-One-Cups, Box of 20 Single-Serve Cups",100 percent pure kona single serve cups for keurig k cup brewing systems exclusive private reserve diamond medium roast kona one cups box of 20 single serve cups,20,0
6,"12 Kone-One-Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",12 kone one cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems,12,0
7,"12 Single-Serve Cups of Water Process Decaf Organic Arabica Coffee, for Use With Keurig K-cup Brewing Systems, From Aloha Island Coffee",12 single serve cups of water process decaf organic arabica coffee for use with keurig k cup brewing systems from aloha island coffee,12,0
8,"12 Kona-One-Cups of Kings Reserve Kona Hawaiian DIAMOND Coffee Blend, Medium Roast",12 kona one cups of kings reserve kona hawaiian diamond coffee blend medium roast,12,0
9,"24 Single-Serve Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",24 single serve cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems,24,0


In [29]:
# visually inspect the outcome to ensure it is correct
df_title_unique.loc[df_title_unique['box'] != 0]

,Title,Title_clean,count,box
15,ARCHER FARMS 18 Single Servings - 2 Pack (Caramel Macchiato),archer farms 18 single servings 2 pack caramel macchiato,18,2
17,Archer Farms K-Cup Cinnamon Vanilla Nut Coffee Light Roast 18 Count (1 Box),archer farms k cup cinnamon vanilla nut coffee light roast 18 count 1 box,18,1
18,"Archer Farms Candy Cane Keurig Coffee, Light Roast- 18 K-Cups (Pack of 2 Boxes)",archer farms candy cane keurig coffee light roast 18 k cups pack of 2 boxes,18,2
27,Barsita Prima Decaf Italian Roast (2 Boxes of 24 K-Cups),barsita prima decaf italian roast 2 boxes of 24 k cups,24,2
36,"Barista Prima Coffeehouse Dark Roast Extra Bold K-Cup for Keurig Brewers, Decaf Italian Roast Coffee,4 pack of 24 cups each (96 count)",barista prima coffeehouse dark roast extra bold k cup for keurig brewers decaf italian roast coffee 4 pack of 24 cups each 96 count,96,4
...,...,...,...,...
4550,Keurig Green Mountain GbPNi Coffee K Cup Packs - Hazelnut Decaf - 72 Count (2 Pack) skYCd,keurig green mountain gbpni coffee k cup packs hazelnut decaf 72 count 2 pack skycd,72,2
4551,Keurig Green Mountain WURBI Coffee K Cup Packs - Breakfast Blend - 72 Count (3 Pack) swxNu,keurig green mountain wurbi coffee k cup packs breakfast blend 72 count 3 pack swxnu,72,3
4553,"Maxwell House Café Collection xpKcG Coffee, Decaf K-Cups, 5.57 oz 18 Count (4 Pack) ZOesK",maxwell house café collection xpkcg coffee decaf k cups 5 57 oz 18 count 4 pack zoesk,18,4
4559,Keurig Green Mountain eRRox Coffee K Cup Packs - French Vanilla - 72 Count (4 Pack) dYdMu,keurig green mountain errox coffee k cup packs french vanilla 72 count 4 pack dydmu,72,4


In [30]:
# values in 'box' column
df_title_unique['box'].unique()

array([  0,   2,   1,   4,  16,  96,  40,  50,  24,   6,  30,  48,  80,
        20,  72,  32,  18,  42,   9,  41,  12,   3,  22,  10,   8,   5,
        88, 144,  65,  36,   7,  44,  66])

In [50]:
# Let's take a look at the box values to ensure we are getting correct value
pd.set_option('display.max_rows', 210)
df_title_unique.loc[df_title_unique['box'] > 6]

,Title,Title_clean,count,box,total
40,Keurig - CafÃ© Escapes CafÃ© Caramel K-Cups (16-Pack),keurig cafã escapes cafã caramel k cups 16 pack,1,16,16
138,"Beantown Roasters K-Cup Variety Pack made up of 11 Artisan Coffees ""No Decaf"" for Keurig, Pack of 96",beantown roasters k cup variety pack made up of 11 artisan coffees no decaf for keurig pack of 96,1,96,96
140,"40 Pack Beantown Roasters Coffee Variety Pack for Keurig K-Cup, You Select The Size. All Coffee ""No Decaf""",40 pack beantown roasters coffee variety pack for keurig k cup you select the size all coffee no decaf,1,40,40
141,"50 Pack Beantown Roasters Coffee Variety Pack for Keurig K-cup, You Select the Size. All Coffee ""No Decaf""",50 pack beantown roasters coffee variety pack for keurig k cup you select the size all coffee no decaf,1,50,50
176,Brooklyn Beans Fuhgeddaboutit KCups - 24ct Box,brooklyn beans fuhgeddaboutit kcups 24ct box,24,24,576
230,"Cafe Bustelo Coffee Espresso, 36-Ounce Cans (Pack of 2) and Dixie to Go PerfecTouch 12 Oz Cups & Lids, 30/Set (Pack of 2)",cafe bustelo coffee espresso 36 ounce cans pack of 2 and dixie to go perfectouch 12 oz cups lids 30 set pack of 2,1,30,30
250,"Café Escapes Café Mocha, K-Cups for Keurig Brewers (Pack of 48)",café escapes café mocha k cups for keurig brewers pack of 48,1,48,48
333,"Caribou Coffee, Caribou Blend, K-Cup Portion Pack for Keurig K-Cup Brewers (Pack of 80)",caribou coffee caribou blend k cup portion pack for keurig k cup brewers pack of 80,1,80,80
337,"Caribou Coffee, Caribou Blend Decaf, K-Cup Portion Pack for Keurig K-Cup Brewers (Pack of 48)",caribou coffee caribou blend decaf k cup portion pack for keurig k cup brewers pack of 48,1,48,48
338,"Caribou Coffee, Daybreak Morning Blend, K-Cup Portion Pack for Keurig K-Cup Brewers (Pack of 48)",caribou coffee daybreak morning blend k cup portion pack for keurig k cup brewers pack of 48,1,48,48


***
**4.3 Total keywords**

total_keywords = ['count', 'ct', 'total', 'net', 'total of' ] 
***

In [32]:
# Since '..total' word has been mentioned the most, we will use it as the main keyword to search for 
# to retrieve numeric value associated with it. If '..total' not found, we will move to other total keywords.

total_list = []

for row in df_title_unique['Title_clean']:
    match = re.search("\d\S*.total", row) 
    if match is not None: 
        match_num = re.search(r'\d+', match.group())
        total_list.append(match_num.group())
    else: 
        match = re.search("total.*\S\d|total of.*\S\d", row)
        if match is not None: 
            match_num = re.search(r'\d+', match.group())
            total_list.append(match_num.group())
        else: 
            total_list.append(0)
            

In [33]:
# create a column 'total' with number of servings associated with total_keywords

pd.set_option('display.max_rows', 250)
df_title_unique['total'] = total_list
df_title_unique['total'] = df_title_unique['total'].astype(str).astype(int)
df_title_unique

,Title,Title_clean,count,box,total
0,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",cake boss coffee k cups variety pack single serve for keurig brewers 36 count,36,0,0
1,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",12 k cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,12,0,0
2,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",12 single serve cups of 100 percent pure kona coffee for use in keurig k cup brewing systems exclusive private reserve diamond kona one cups medium roast box of 12 cups,12,0,0
3,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",24 single serve kona one cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,24,0,0
4,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",24 single serve kona one cups of water process decaf organic arabica tropical coffee for keurig k cup brewing systems from aloha island coffee,24,0,0
...,...,...,...,...,...
4567,"Italian Roast K-Cup Variety Pack, Keurig 2.0 K Cup Coffee Sampler with Adagio Toscana Blend, Faro Italian, Lavazza Gran Selezione & Tully's Italian Roast Extra Bold, Keurig 2.0 K-Cups (96 Count)",italian roast k cup variety pack keurig 2 0 k cup coffee sampler with adagio toscana blend faro italian lavazza gran selezione tully s italian roast extra bold keurig 2 0 k cups 96 count,96,0,0
4568,"Green Mountain Coffee WccIm K-Cup Portion Count for Keurig K-Cup Brewers, Caramel Vanilla Cream, 24 Count (Pack of 2) GvlSZ",green mountain coffee wccim k cup portion count for keurig k cup brewers caramel vanilla cream 24 count pack of 2 gvlsz,24,0,0
4569,Keurig Green Mountain HzrVf Coffee K Cup Packs - Breakfast Blend - 72 Count (2 Pack) IYcfv,keurig green mountain hzrvf coffee k cup packs breakfast blend 72 count 2 pack iycfv,72,2,0
4570,"Green Mountain Coffee ixILa K-Cup Portion Count for Keurig K-Cup Brewers, Caramel Vanilla Cream, 24 Count (Pack of 2) QOBEQ",green mountain coffee ixila k cup portion count for keurig k cup brewers caramel vanilla cream 24 count pack of 2 qobeq,24,0,0


In [34]:
# Let's take a look at the output to visually inspect it. 
df_title_unique.loc[df_title_unique['total'] != 0]

,Title,Title_clean,count,box,total
215,Café Bustelo Espresso Style K-Cup Pods for Keurig K-Cup Brewers 6 boxes of 12 (72 total),café bustelo espresso style k cup pods for keurig k cup brewers 6 boxes of 12 72 total,0,6,72
753,"Don Francisco's Variety Pack (Breakfast Blend, French Roast, Kona Blend, and Vanilla Nut), Premium 100% Arabica Coffee Beans, Single Serve Pods for Keurig, Family Reserve, 4X12 Count (48 Total)",don francisco s variety pack breakfast blend french roast kona blend and vanilla nut premium 100 percent arabica coffee beans single serve pods for keurig family reserve 4x12 count 48 total,4,0,48
754,"Don Francisco's French Roast, Rich Premium 100% Arabica Coffee Beans, Dark Roast, Ground, Single Serve Pods for Keurig, Family Reserve, 18-Count Combo Pack (36 Total)",don francisco s french roast rich premium 100 percent arabica coffee beans dark roast ground single serve pods for keurig family reserve 18 count combo pack 36 total,18,0,36
755,"Don Francisco's Breakfast Blend, Premium 100% Arabica Coffee Beans, Medium-Dark Roast, Ground, Single Serve Pods for Keurig, Family Reserve, 18-Count Combo Pack (36 Total)",don francisco s breakfast blend premium 100 percent arabica coffee beans medium dark roast ground single serve pods for keurig family reserve 18 count combo pack 36 total,18,0,36
756,"Don Francisco's Dark Roast Variety Pack (Breakfast Blend and French Roast Flavored Coffee), Single Serve Pods for Keurig, Family Reserve, 18-Count (36 Total)",don francisco s dark roast variety pack breakfast blend and french roast flavored coffee single serve pods for keurig family reserve 18 count 36 total,18,0,36
757,"Don Francisco's 100% Colombia Supremo, Premium 100% Arabica Coffee Beans, Medium Roast, Single Serve Pods for Keurig, Family Reserve, 6 -12 Count (72 Total)",don francisco s 100 percent colombia supremo premium 100 percent arabica coffee beans medium roast single serve pods for keurig family reserve 6 12 count 72 total,12,0,72
759,"Don Francisco's Hawaiian Hazelnut, Premium 100% Arabica, Flavored Coffee, Medium Roast, Single-Serve Pods, Family Reserve, 12-Count (Pack of 6, 72 Total)",don francisco s hawaiian hazelnut premium 100 percent arabica flavored coffee medium roast single serve pods family reserve 12 count pack of 6 72 total,12,0,72
760,"Don Francisco's Blueberry, Premium 100% Arabica Coffee, Flavored Coffee, Mediim Roast, Single Serve Pods for Keurig, 6 -12 Count (72 Total)",don francisco s blueberry premium 100 percent arabica coffee flavored coffee mediim roast single serve pods for keurig 6 12 count 72 total,12,0,72
766,"Don Francisco's Cinnamon Hazelnut, Premium 100% Arabica Coffee Beans, Medium Roast, Ground, Single Serve Pods for Keurig, Family Reserve, 12-Count, Pack of 6 (72 Total)",don francisco s cinnamon hazelnut premium 100 percent arabica coffee beans medium roast ground single serve pods for keurig family reserve 12 count pack of 6 72 total,12,0,72
767,"Don Francisco's Organic Mayan, Premium 100% Arabica Coffee Beans, Medium-Dark Roast, Single Serve Pods for Keurig, Family Reserve, 6 -12 Count (72 Total)",don francisco s organic mayan premium 100 percent arabica coffee beans medium dark roast single serve pods for keurig family reserve 6 12 count 72 total,12,0,72


In [35]:
# unique values in 'total' column
df_title_unique['total'].unique()

array([  0,  72,  48,  36,   3,   2, 160,  60,  10,  96, 192,  40,  24,
         6,  88])

In [36]:
df_title_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        4572 non-null   string
 1   Title_clean  4572 non-null   string
 2   count        4572 non-null   int64 
 3   box          4572 non-null   int64 
 4   total        4572 non-null   int64 
dtypes: int64(3), string(2)
memory usage: 178.7 KB


*** 
**5. Calculating Total Servings**
***

In [37]:
# replace 'box' and 'count' value 0 with 1 if the 'title' data doesn't explicitly states number of boxes or 
# single-cup count, we assume 1 box or 1 serving
df_title_unique['box'].replace(0, 1, inplace = True)
df_title_unique['count'].replace(0, 1, inplace = True)
df_title_unique.head(10)

,Title,Title_clean,count,box,total
0,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",cake boss coffee k cups variety pack single serve for keurig brewers 36 count,36,1,0
1,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",12 k cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,12,1,0
2,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",12 single serve cups of 100 percent pure kona coffee for use in keurig k cup brewing systems exclusive private reserve diamond kona one cups medium roast box of 12 cups,12,1,0
3,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",24 single serve kona one cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,24,1,0
4,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",24 single serve kona one cups of water process decaf organic arabica tropical coffee for keurig k cup brewing systems from aloha island coffee,24,1,0
5,"100% Pure Kona Single-Serve Cups for Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Medium Roast Kona-One-Cups, Box of 20 Single-Serve Cups",100 percent pure kona single serve cups for keurig k cup brewing systems exclusive private reserve diamond medium roast kona one cups box of 20 single serve cups,20,1,0
6,"12 Kone-One-Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",12 kone one cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems,12,1,0
7,"12 Single-Serve Cups of Water Process Decaf Organic Arabica Coffee, for Use With Keurig K-cup Brewing Systems, From Aloha Island Coffee",12 single serve cups of water process decaf organic arabica coffee for use with keurig k cup brewing systems from aloha island coffee,12,1,0
8,"12 Kona-One-Cups of Kings Reserve Kona Hawaiian DIAMOND Coffee Blend, Medium Roast",12 kona one cups of kings reserve kona hawaiian diamond coffee blend medium roast,12,1,0
9,"24 Single-Serve Cups of Dark Roast, Kona Smooth Kings Reserve Kona Hawaiian Blend Coffee, for Keurig K-cup Brewing Systems",24 single serve cups of dark roast kona smooth kings reserve kona hawaiian blend coffee for keurig k cup brewing systems,24,1,0


In [38]:
for row in df_title_unique['total']:
    if row < 4: 
        df_title_unique['total'] = df_title_unique['count'] * df_title_unique['box']
    else:
        pass

In [39]:
 df_title_unique

,Title,Title_clean,count,box,total
0,"Cake Boss Coffee K Cups Variety Pack, Single Serve For Keurig Brewers 36 Count",cake boss coffee k cups variety pack single serve for keurig brewers 36 count,36,1,36
1,"12 K-cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",12 k cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,12,1,12
2,"12 Single-Serve Cups of 100% Pure Kona Coffee for Use in Keurig K-cup Brewing Systems, Exclusive Private Reserve Diamond Kona-One-Cups, Medium Roast, Box of 12 Cups",12 single serve cups of 100 percent pure kona coffee for use in keurig k cup brewing systems exclusive private reserve diamond kona one cups medium roast box of 12 cups,12,1,12
3,"24 Single-Serve Kona-One-Cups of Breakfast Blend Certified Organic Arabica Tropical Coffee, Surf's Up! Get-your-day-started Breakfast Blend",24 single serve kona one cups of breakfast blend certified organic arabica tropical coffee surf s up get your day started breakfast blend,24,1,24
4,"24 Single-Serve Kona-One-Cups of Water Process Decaf Organic Arabica Tropical Coffee, for Keurig K-cup Brewing Systems, From Aloha Island Coffee",24 single serve kona one cups of water process decaf organic arabica tropical coffee for keurig k cup brewing systems from aloha island coffee,24,1,24
...,...,...,...,...,...
4567,"Italian Roast K-Cup Variety Pack, Keurig 2.0 K Cup Coffee Sampler with Adagio Toscana Blend, Faro Italian, Lavazza Gran Selezione & Tully's Italian Roast Extra Bold, Keurig 2.0 K-Cups (96 Count)",italian roast k cup variety pack keurig 2 0 k cup coffee sampler with adagio toscana blend faro italian lavazza gran selezione tully s italian roast extra bold keurig 2 0 k cups 96 count,96,1,96
4568,"Green Mountain Coffee WccIm K-Cup Portion Count for Keurig K-Cup Brewers, Caramel Vanilla Cream, 24 Count (Pack of 2) GvlSZ",green mountain coffee wccim k cup portion count for keurig k cup brewers caramel vanilla cream 24 count pack of 2 gvlsz,24,1,24
4569,Keurig Green Mountain HzrVf Coffee K Cup Packs - Breakfast Blend - 72 Count (2 Pack) IYcfv,keurig green mountain hzrvf coffee k cup packs breakfast blend 72 count 2 pack iycfv,72,2,144
4570,"Green Mountain Coffee ixILa K-Cup Portion Count for Keurig K-Cup Brewers, Caramel Vanilla Cream, 24 Count (Pack of 2) QOBEQ",green mountain coffee ixila k cup portion count for keurig k cup brewers caramel vanilla cream 24 count pack of 2 qobeq,24,1,24


**6. Merging dataframes to get total servings count per SKU / weekly order**

In [41]:
# merge dataframes df and df_title_unique on 'Title' column
df_total_servings = df.merge(df_title_unique, how = 'left', on = 'Title')

In [42]:
df_total_servings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157499 entries, 0 to 157498
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SKU              157499 non-null  string 
 1   Title            157499 non-null  string 
 2   Brand            157499 non-null  string 
 3   Parent Category  157499 non-null  object 
 4   Sub-Category     157499 non-null  object 
 5   WeekID           157499 non-null  int64  
 6   Retail Price     157499 non-null  float64
 7   Sales            157499 non-null  float64
 8   Title_clean_x    157499 non-null  string 
 9   Title_clean_y    157499 non-null  string 
 10  count            157499 non-null  int64  
 11  box              157499 non-null  int64  
 12  total            157499 non-null  int64  
dtypes: float64(2), int64(4), object(2), string(5)
memory usage: 16.8+ MB


In [43]:
# exporting dataframe to excel

df_total_servings.to_excel('assessment.xlsx', sheet_name = 'data')